* gammas from traces (not generated via predictions!!) 
    * save (∂format) & load in-
    * -> correlate across format

In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path  as op



In [4]:
frac = np.linspace(-6,6, 13)
frac = np.delete(frac, 6)                     
fractions = np.power(2,(frac/4))

base = np.array([5, 7, 10, 14, 20])

In [5]:
fractions

array([0.35355339, 0.42044821, 0.5       , 0.59460356, 0.70710678,
       0.84089642, 1.18920712, 1.41421356, 1.68179283, 2.        ,
       2.37841423, 2.82842712])

In [7]:
12 * 5 * 3

180

In [ ]:
import arviz as az

model_label = 4 # 1 '1rem1' #3 
format = 'symbolic' # 'both' # 'non-symbolic' # 

traces = az.from_netcdf( op.join(target_folder, f'rnp_model-{model_label}_format-{format}_trace.netcdf'))


In [ ]:
## Get params
#(from traces directly, not vie predict, maybe a bad idea?- but extract_rnp_prec stopped working at some point)

target_folder = op.join(bids_folder, 'derivatives/phenotype')

dys_subs = group_list[group_list['group']==1]['subject'].values.tolist()
dys_subs = [str(x) for x in dys_subs]   
con_subs = group_list[group_list['group']==0]['subject'].values.tolist()
con_subs = [str(x) for x in con_subs] 

gammas_sub = traces.posterior['x|subject'].to_dataframe() #.groupby('subject').mean().mean()
gammas_int = traces.posterior['x'].to_dataframe()
gammas_group = traces.posterior['x:group'].to_dataframe()

gammas = gammas_sub.join(gammas_int, on=['chain','draw']).join(gammas_group,on=['chain','draw'])
gammas['sum_dys']= gammas['x'] +  gammas['x|subject'] +  gammas['x:group'] 
gammas['sum_con']= gammas['x'] +  gammas['x|subject']
gammas_d = gammas['sum_dys'].unstack([0,1]).mean(axis=1).to_frame().loc[dys_subs]
gammas_c =  gammas['sum_con'].unstack([0,1]).mean(axis=1).to_frame().loc[con_subs]
gammas = pd.concat([gammas_c,gammas_d]).reset_index().rename(columns={0:'gamma','subject__factor_dim':'subject'})
gammas.to_csv(op.join(target_folder, f'probit_model-{model_label}_format-{format}_gammas.csv'))
gammas.head()

In [ ]:
stake_sub = traces.posterior['C(n_safe)|subject'].to_dataframe() #.groupby('subject').mean().mean()
stake_int = traces.posterior['C(n_safe)'].to_dataframe()
stake_group = traces.posterior['group:C(n_safe)'].to_dataframe()

In [ ]:
stake = stake_sub.join(stake_int).join(stake_group)
stake['sum_dys']= stake['C(n_safe)'] +  stake['C(n_safe)|subject'] +  stake['group:C(n_safe)'] 
stake['sum_con']= stake['C(n_safe)'] +  stake['C(n_safe)|subject']
stake_d = stake['sum_dys'].unstack([0,1]).mean(axis=1).to_frame().loc[dys_subs]
stake_c =  stake['sum_con'].unstack([0,1]).mean(axis=1).to_frame().loc[con_subs]
stake = pd.concat([stake_c,stake_d]).reset_index().rename(columns={0:'gamma','subject__factor_dim':'subject'})
stake.to_csv(op.join(target_folder, f'probit_model-{model_label}_format-{format}_stake-sizes.csv'))
stake.head()

In [ ]:
# read in & correlate
target_folder = op.join(bids_folder,  'derivatives/phenotype')

gammas_symbolic = pd.read_csv(op.join(target_folder, f'probit_model-2_format-symbolic_gammas.csv'))
gammas_symbolic = gammas_symbolic.set_index('subject').drop(columns=['Unnamed: 0'])
gammas_nonsymbolic = pd.read_csv(op.join(target_folder, f'probit_model-2_format-non-symbolic_gammas.csv'))
gammas_nonsymbolic = gammas_nonsymbolic.set_index('subject').drop(columns=['Unnamed: 0'])

group_list['group'] = np.where(group_list['group'] == 0, 'control', 'dyscalc')

gammas = gammas_symbolic.join(gammas_nonsymbolic, lsuffix='_symbolic', rsuffix='_nonsymbolic').join(group_list.set_index('subject'))
gammas.head()

In [ ]:
sns.lmplot(data=gammas, x='gamma_symbolic', y='gamma_nonsymbolic', hue='group', fit_reg=True)

## old

false with joining datframes etc., predict method way easier !

In [ ]:
par_gr_n = traces.posterior['x:group:C(n_safe)'].to_dataframe() # for model-4 format-symbolic
par_gr_n = par_gr_n
par_n = traces.posterior['x:C(n_safe)'].to_dataframe()
par_x = traces.posterior['x'].to_dataframe()
par_gr = traces.posterior['x:group'].to_dataframe()

par = par_n.join(par_gr_n.droplevel(2), on=['chain','draw']).join(par_x, on=['chain','draw']).join(par_gr, on=['chain','draw'])
#par.head()
par['control'] = par['x'] + par['x:C(n_safe)']
par['dyscalc'] = par['x'] + par['x:C(n_safe)'] + par['x:group'] + par['x:group:C(n_safe)'] 
par = par[['control','dyscalc']]

par = par.stack().to_frame()
par = par.reset_index().rename(mapper={'level_3':'group','x:C(n_safe)_dim':'n_safe',0:'gamma' },axis=1).set_index(['chain','draw','n_safe','group'])
par.head()

In [ ]:
sns.violinplot(data=par.reset_index(), x='n_safe', y='gamma', hue='group')

for n in par.index.unique('n_safe').values :
    print(n)
    temp = par.xs(n, 0,level='n_safe')
    diff = np.mean((temp.xs('dyscalc', level='group') - temp.xs('control', level='group') ) > 0 )
    print(diff)

In [ ]:
reg = 'x:C(n_safe)' # 'x:group:C(n_safe)'
n_safe_level = f'{reg}_dim'
data = par_n # par_gr_n

sns.violinplot(data=data.reset_index(), x=n_safe_level, y=reg)
plt.axhline(0,c='k',)
plt.title(reg)

for n_safe in data.index.unique(n_safe_level):
    p_val = np.round(np.mean(data.xs(n_safe,0,n_safe_level)>0),3)
    print(p_val)
#par_gr_n